<a href="https://colab.research.google.com/github/Mozzer2310/COMP34711-Deep-Learning/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import random
import string
from nltk.tokenize import word_tokenize
import tensorflow as tf
import numpy as np


class NeuralNetwork:

    def __init__(self) -> None:
        self.vocab = set()
        self.reviews = []
        self.classification = []

    def read_data(self, path: str) -> list:
        # Find all the .txt files at the path, remove the README from the list
        file_paths = glob.glob(path + "/*.txt")
        file_paths.remove(path + "/README.txt")

        corpora = []
        # Read each file in the list of files
        for file_path in file_paths:
            f = open(file_path, "r")
            # Add the data to an array of corpora
            corpora.append(f.read())

        return corpora

    def preprocess(self, corpora: list):
        self.reviews = []
        self.classification = []
        # process the raw data of each corpus in the list
        for corpus in corpora:
            self.process_raw(corpus)

        # Flatten the processed reviews, to get a single list, convert to a set to get the vocab list
        self.vocab = set(
            [item for sublist in self.reviews for item in sublist.split(" ")])

    def process_raw(self, raw: str):
        # split over the lines (## defines a line and is on each new line as defined by README)
        lines = raw.splitlines()
        # remove '[t]' tags
        lines = [ele for ele in lines if ele != "[t]"]

        # process each line in the text, add the result to an array and add review class to an array
        for line in lines:
            # Check that the line isn't empty
            if len(line) != 0:
                # Process the line, get returned processed line and its review info for classifying
                processed_review, review_info = self.process_line(line)
                # Only consider reviews which can be classified, i.e. have been classified in text file
                if len(review_info) != 0:
                    # Consider weights of reviews, in the case that a review is part positive and part negative
                    # the weights will help when classifying a review if it is 'more' postive than negative, and vice versa
                    num_pos = 3 * review_info.count("+3") + 2 * review_info.count(
                        "+2") + review_info.count("+1") + review_info.count("+")
                    num_neg = 3 * review_info.count("-3") + 2 * review_info.count(
                        "-2") + review_info.count("-1") + review_info.count("-")
                    # 1 for postive and 0 for negative review, add to list
                    if num_pos > num_neg:
                        self.classification.append(1)
                        # add the review to an array
                        self.reviews.append(processed_review)
                    elif num_pos < num_neg:
                        self.classification.append(0)
                        # add the review to an array
                        self.reviews.append(processed_review)

    def process_line(self, line: str):
        # Get the substring before the ## delimiter, if not present return empty values for error handling
        try:
            delim_index = line.index("##")
        except ValueError:
            delim_index = None
        if delim_index == None:
            return [], ""
        # sub-string before the delimiter is the information about the class of review
        review_info = line[:delim_index]
        # sub-string after the delimiter is the review
        line = line[delim_index+2:]

        # Convert to lower case
        line_lwr = line.lower()
        # # Remove everything except punctuation
        line_clean = line_lwr.translate(
            str.maketrans('', '', string.punctuation))
        # Tokenize the review and rejoin to remove whitespace
        line_tokens = " ".join(word_tokenize(line_clean))

        return line_tokens, review_info

    def test(self, train_data: list, train_class: list, test_data: list, test_class: list):
        ######### USING ENCODER ######################
        train_dataset = tf.data.Dataset.from_tensor_slices(
            (train_data, train_class))
        test_dataset = tf.data.Dataset.from_tensor_slices(
            (test_data, test_class))

        BUFFER_SIZE = 10000
        BATCH_SIZE = 32
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(
            BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        test_dataset = test_dataset.batch(
            BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

        VOCAB_SIZE = 5000
        encoder = tf.keras.layers.TextVectorization(
            max_tokens=VOCAB_SIZE)
        encoder.adapt(train_dataset.map(lambda text, label: text))

        model = tf.keras.Sequential([
            encoder,
            tf.keras.layers.Embedding(
                input_dim=len(encoder.get_vocabulary()),
                output_dim=32,
                # Use masking to handle the variable sequence lengths
                mask_zero=True),
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(32, recurrent_dropout=0.2)),
            # tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='relu')
        ])

        model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      optimizer=tf.keras.optimizers.Adam(1e-4),
                      metrics=['accuracy'])

        history = model.fit(
            train_dataset,
            epochs=10,
            batch_size=BATCH_SIZE,
            verbose=0)

        test_loss, test_acc = model.evaluate(test_dataset, verbose=0)

        return test_acc

    def nfold_cv(self, n: int = 5):
        pos_inds = np.where(np.array(self.classification) == 1)
        neg_inds = np.where(np.array(self.classification) == 0)

        pos_reviews = list(np.array(self.reviews)[pos_inds])
        neg_reviews = list(np.array(self.reviews)[neg_inds])

        shuffled_pos = pos_reviews.copy()
        shuffled_neg = neg_reviews.copy()
        random.shuffle(shuffled_pos)
        random.shuffle(shuffled_neg)

        groups = {"test": [], "test_class": [], "train": [], "train_class": []}
        for split in range(n):
            lower_pos = (len(shuffled_pos)//n) * split
            lower_neg = (len(shuffled_neg)//n) * split
            if split == n-1:
                test_pos = shuffled_pos[lower_pos:]
                test_neg = shuffled_neg[lower_neg:]
                train_pos = shuffled_pos[:lower_pos]
                train_neg = shuffled_neg[:lower_neg]
            else:
                upper_pos = (len(shuffled_pos)//n) * (split+1)
                upper_neg = (len(shuffled_neg)//n) * (split+1)
                test_pos = shuffled_pos[lower_pos:upper_pos]
                test_neg = shuffled_neg[lower_neg:upper_neg]
                train_pos = shuffled_pos[:lower_pos] + shuffled_pos[upper_pos:]
                train_neg = shuffled_neg[:lower_neg] + shuffled_neg[upper_neg:]
            groups["test"].append(test_pos + test_neg)
            groups["test_class"].append(
                [1] * len(test_pos) + [0] * len(test_neg))
            groups["train"].append(train_pos + train_neg)
            groups["train_class"].append(
                [1] * len(train_pos) + [0] * len(train_neg))

        # print(groups)
        accuracies = []
        for fold in range(n):
            accuracies.append(self.test(
                groups["train"][fold], groups["train_class"][fold], groups["test"][fold], groups["test_class"][fold]))

        print(accuracies)


2022-12-08 17:48:41.828103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 17:48:42.323026: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-12-08 17:48:42.323077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-12-08 17:48:42.323081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning

In [2]:
def main():
    neural = NeuralNetwork()
    # specify the directory path to the review files
    corpora = neural.read_data("product_reviews")

    neural.preprocess(corpora)
    neural.nfold_cv()


test = main()


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2022-12-08 17:48:43.251913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 17:48:43.268285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-12-08 17:48:43.268306: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-12-08 17:48:43.268572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary 

[0.7440191507339478, 0.7153109908103943, 0.6961722373962402, 0.7559808492660522, 0.7156398296356201]
